### Data Transformation

In [1]:
import os
%pwd

'c:\\Users\\alok4\\Desktop\\Text-Summarizer\\research'

In [2]:
os.chdir('..')

In [3]:
%pwd

'c:\\Users\\alok4\\Desktop\\Text-Summarizer'

In [17]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [18]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)

        create_directories([self.config.artifact_root],verbose=True)

    def get_data_transformation_config(self)->DataTransformationConfig:
        config=self.config.data_transformation
        create_directories([config.root_dir],verbose=True)
        data_transformation_config=DataTransformationConfig(root_dir=config.root_dir,data_path=config.data_path,tokenizer_name=config.tokenizer_name)
        return data_transformation_config

In [20]:
import os
from src.textSummarizer.logging import logging
from transformers import AutoTokenizer
from datasets import load_from_disk

### Data Transformation Component

In [21]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config=config
        self.tokenizer=AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings=self.tokenizer(example_batch['summary'],max_length=128,truncation=True)
        return {
            'input_ids':input_encodings['input_ids'],
            'attention_mask':input_encodings['attention_mask'],
            'labels':target_encodings['input_ids']
        }
    def convert(self):
        data_samsum=load_from_disk(self.config.data_path)
        data_samsum_pt=data_samsum.map(self.convert_examples_to_features,batched=True)
        data_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_datset"))

In [23]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2025-08-28 23:59:24,167:INFO:common:Yaml file:config\config.yaml loaded successfully]
[2025-08-28 23:59:24,169:INFO:common:Yaml file:params.yaml loaded successfully]
[2025-08-28 23:59:24,170:INFO:common:Created Directory at: artifacts]
[2025-08-28 23:59:24,170:INFO:common:Created Directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 159697.48 examples/s]
